The gradients computed in the `contrastive_divergence()` function using `torch.mm()` are **outer products**, and this is a key concept in training Restricted Boltzmann Machines (RBMs). Let me break it down step by step:

---

### 1. **What is `torch.mm()` Doing?**
`torch.mm(A, B)` performs a **matrix multiplication** between two tensors `A` and `B`. In this case:
- `positive_grad = torch.mm(h_sample.t(), v)` computes the outer product of the hidden activations (`h_sample`) and the visible units (`v`) during the **positive phase**.
- `negative_grad = torch.mm(p_h_given_v_sample.t(), v_sample)` computes the outer product of the hidden activations (`p_h_given_v_sample`) and the reconstructed visible units (`v_sample`) during the **negative phase**.

---

### 2. **Why Outer Products?**
The **outer product** is used because the weight matrix `W` in an RBM connects every visible unit to every hidden unit. The weight update rule in RBMs is based on the correlations between visible and hidden units. Specifically:
- The **positive phase** captures the correlation between the visible units (`v`) and the hidden units (`h_sample`) when the model is exposed to the real data.
- The **negative phase** captures the correlation between the reconstructed visible units (`v_sample`) and the hidden units (`p_h_given_v_sample`) after Gibbs sampling.

The outer product is the natural way to compute these correlations because:
- The outer product of a column vector (e.g., `h_sample`) and a row vector (e.g., `v`) results in a matrix where each element represents the interaction (or correlation) between a specific hidden unit and a specific visible unit.

---

### 3. **Positive and Negative Gradients**
- **Positive Gradient (`positive_grad`)**:
  - This is computed using the real data (`v`) and the hidden activations (`h_sample`).
  - It represents how the weights should be adjusted to increase the probability of the observed data.
  - It is called "positive" because it reinforces the correlations between visible and hidden units for the real data.

- **Negative Gradient (`negative_grad`)**:
  - This is computed using the reconstructed data (`v_sample`) and the hidden activations (`p_h_given_v_sample`).
  - It represents how the weights should be adjusted to decrease the probability of the reconstructed data (which the model generates itself).
  - It is called "negative" because it counteracts the model's tendency to overfit to its own reconstructions.

---

### 4. **Why Does This Work?**
The goal of training an RBM is to adjust the weights (`W`) such that the model assigns higher probabilities to the real data and lower probabilities to the reconstructed data. This is achieved by:
- **Maximizing the log-likelihood** of the data, which involves computing the gradient of the log-likelihood with respect to the weights.
- The gradient of the log-likelihood can be approximated using **Contrastive Divergence (CD)**, which uses the difference between the positive and negative gradients:
  ```python
  ΔW = learning_rate * (positive_grad - negative_grad) / batch_size
  ```
  This update rule ensures that the weights are adjusted to better represent the real data while discouraging the model from overfitting to its own reconstructions.

---

### 5. **Why Outer Products Specifically?**
The weight matrix `W` in an RBM connects every visible unit to every hidden unit. The outer product is used because:
- Each weight `W[i, j]` represents the connection between visible unit `i` and hidden unit `j`.
- The outer product computes all these pairwise interactions (correlations) at once, resulting in a matrix of the same shape as `W`.

For example:
- If `v` has shape `(batch_size, visible_dim)` and `h_sample` has shape `(batch_size, hidden_dim)`, then:
  - `h_sample.t()` has shape `(hidden_dim, batch_size)`.
  - `torch.mm(h_sample.t(), v)` results in a matrix of shape `(hidden_dim, visible_dim)`, which matches the shape of `W`.

---

### 6. **Summary**
- The **positive gradient** captures correlations between real data and hidden units, reinforcing the model's ability to represent the data.
- The **negative gradient** captures correlations between reconstructed data and hidden units, discouraging overfitting to the model's own reconstructions.
- Both gradients are computed as **outer products** because the weight matrix `W` connects every visible unit to every hidden unit, and the outer product naturally captures these pairwise interactions.
- The weight update rule (`W += learning_rate * (positive_grad - negative_grad)`) ensures the model learns to better represent the real data while avoiding overfitting.

This is why the outer product and the distinction between positive and negative gradients are central to training RBMs.

# Midterm 2, Assignment 3 - Gaetano Barresi [579102]

A Restricted Boltzmann Machine (RBM) is a generative stochastic neural network that learns a probability distribution over its inputs. It is widely used for unsupervised learning, dimensionality reduction, and feature extraction.
To implement from scratch a RBM we must define first its architecture. It is a simple two layer neural network, one input layer (visible states, our data) and one hidden layer (the hidden states, latent feature representation). For parameters, we have a set of weights and two set of bias, one for the visible units and one for the hidden units:


```python
self.W = torch.randn(hidden_dim, visible_dim, device=self.device) * 0.01
self.v_bias = torch.zeros(visible_dim, device=self.device)
self.h_bias = torch.zeros(hidden_dim, device=self.device)
```


Weights W are initialized with small values and biases with zeros.
Hidden units are conditionally independent given visible units and viceversa, due to not oriented edges and bipartition structure:


$$
\
\mathbb{P}(h_j \mid v) = \sigma \left( \sum_i M_{ij} v_i + c_j \right) \quad \forall j
\
$$


$$
\
\mathbb{P}(v_i \mid h) = \sigma \left( \sum_j M_{ij} h_j + b_i \right) \quad \forall i
\
$$


These are resepctively the forward pass (wake) and the backward pass (dream) and they can be implemented as:


```python
def visible_to_hidden(self, v):  # forward pass
    # compute probabilities of hidden units given visible units
    p_h = torch.sigmoid(F.linear(v, self.W, self.h_bias))
    return p_h
    
def hidden_to_visible(self, h):  # backward pass
    # compute probabilities of visible units given hidden units
    p_v = torch.sigmoid(F.linear(h, self.W.t(), self.v_bias))
    return p_v
```


In a RBM, sampling is a key step in the training process, particularly during Gibbs sampling. In order to generate binary samples (0 or 1) from a given probability distribution p, we use the following function. These binary samples represent the activation states of the visible or hidden units in the RBM.


```python
def sample_from_p(self, p):
    # bernoulli sampling given probabilities
    return F.relu(torch.sign(p - torch.rand_like(p, device=self.device)))
```


`sample_from_p` takes p, which is a tensor of probabilities. Each value in p represents the probability of a unit being activated (set to 1).
`torch.rand_like(p)` generates a tensor of random values, uniformly distributed between 0 and 1, with the same shape as p. `p - torch.rand_like(p)` computes the difference between the probabilities and the random values.
`torch.sign(p - torch.rand_like(p))` produces a tensor where values greater than 0 are set to 1, and values less than or equal to 0 are set to -1. This effectively performs a thresholding operation to decide whether each unit is activated (1) or not (-1).
`F.relu(...)` ensures that all negative values are clamped to 0. This step converts the -1 values to 0, resulting in a binary tensor of 0s and 1s.
All these pieces are used inside the generalized version of Contrastive Divergence (CD) learning algorithm. It is divided in positive phase (wake part), negative phase(dream part) and parameters update.
Positive phase computes the hidden probabilities (`p_h_given_v`) and sample hidden states (`h_sample`). It computes also the positive gradient as the outer product of `h_sample` and `v`.


```python
# positive phase
p_h_given_v = self.visible_to_hidden(v)
h_sample = self.sample_from_p(p_h_given_v)
positive_grad = torch.mm(h_sample.t(), v)
```


Negative phase performs k steps of Gibbs sampling to reconstruct the visible and hidden states and computes the negative gradient as the outer product of the reconstructed hidden probabilities and visible states.


```python
# gibbs sampling (negative phase)
v_sample = v
for _ in range(self.k):
    p_h_given_v = self.visible_to_hidden(v_sample)
    h_sample = self.sample_from_p(p_h_given_v)
    p_v_given_h = self.hidden_to_visible(h_sample)
    v_sample = self.sample_from_p(p_v_given_h)

# negative phase
p_h_given_v_sample = self.visible_to_hidden(v_sample)
negative_grad = torch.mm(p_h_given_v_sample.t(), v_sample)
```


Weights (`W`) and biases (`v_bias`, `h_bias`) are updated using the difference between the positive and negative gradients, normalized by the batch size.


```python
self.W += self.learning_rate * (positive_grad - negative_grad) / v.size(0)
self.v_bias += self.learning_rate * torch.sum(v - v_sample, dim=0) / v.size(0)
self.h_bias += self.learning_rate * torch.sum(p_h_given_v - p_h_given_v_sample, dim=0) / v.size(0)
```


We now pack all this stuff in a custom class `RBM`, provided with its own training method and train two different RBMs, with different values of k for CD algorithm: k=4 and k=8.

In [ ]:
import torch
import torch.nn.functional as F
from tqdm import tqdm

class RBM:
    def __init__(self, visible_dim, hidden_dim, k=1, lr=0.01, device='cpu'):
        self.visible_dim = visible_dim
        self.hidden_dim = hidden_dim
        self.k = k  # number of Gibbs sampling steps
        self.learning_rate = lr
        self.device = device  # Device to run the computations on (e.g., 'cuda' or 'cpu')
        
        # weights and biases initialization
        self.W = torch.randn(hidden_dim, visible_dim, device=self.device) * 0.01
        self.v_bias = torch.zeros(visible_dim, device=self.device)
        self.h_bias = torch.zeros(hidden_dim, device=self.device)
    
    def sample_from_p(self, p):
        # bernoulli sampling given probabilities
        return F.relu(torch.sign(p - torch.rand_like(p, device=self.device)))
    
    def visible_to_hidden(self, v):  # forward pass
        # compute probabilities of hidden units given visible units
        p_h = torch.sigmoid(F.linear(v, self.W, self.h_bias))
        return p_h
    
    def hidden_to_visible(self, h):  # backward pass
        # compute probabilities of visible units given hidden units
        p_v = torch.sigmoid(F.linear(h, self.W.t(), self.v_bias))
        return p_v

    def contrastive_divergence(self, v):
        # Move input to the correct device
        v = v.to(self.device)

        # positive phase
        p_h_given_v = self.visible_to_hidden(v)
        h_sample = self.sample_from_p(p_h_given_v)
        positive_grad = torch.mm(h_sample.t(), v)

        # gibbs sampling (negative phase)
        v_sample = v
        for _ in range(self.k):
            p_h_given_v = self.visible_to_hidden(v_sample)
            h_sample = self.sample_from_p(p_h_given_v)
            p_v_given_h = self.hidden_to_visible(h_sample)
            v_sample = self.sample_from_p(p_v_given_h)

        # negative phase
        p_h_given_v_sample = self.visible_to_hidden(v_sample)
        negative_grad = torch.mm(p_h_given_v_sample.t(), v_sample)

        # update weights and biases
        self.W += self.learning_rate * (positive_grad - negative_grad) / v.size(0)
        self.v_bias += self.learning_rate * torch.sum(v - v_sample, dim=0) / v.size(0)
        self.h_bias += self.learning_rate * torch.sum(p_h_given_v - p_h_given_v_sample, dim=0) / v.size(0)

    def train(self, data_loader, epochs=10):
        for epoch in range(epochs):
            epoch_error = 0
            for batch in tqdm(data_loader, desc="Training Batches", leave=False):
                # Extract data from batch (ignore labels)
                batch, _ = batch  # Unpack the tuple (data, labels)
                batch = batch.view(-1, self.visible_dim).to(self.device)  # Flatten input and move to device
                self.contrastive_divergence(batch)
                
                # Compute reconstruction error
                v_reconstructed = self.hidden_to_visible(self.sample_from_p(self.visible_to_hidden(batch)))
                epoch_error += torch.sum((batch - v_reconstructed) ** 2).item()
            
            print(f"Epoch {epoch + 1}/{epochs}, Reconstruction Error: {epoch_error}")

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import TensorDataset, DataLoader


visible_dim = 784  # For MNIST
hidden_dim = 256  # Number of hidden neurons in RBM
num_classes = 10  # Digits 0-9

# Use GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Initialize RBMs
rbm4 = RBM(visible_dim, hidden_dim, k=4, lr=0.01, device=device)
rbm8 = RBM(visible_dim, hidden_dim, k=8, lr=0.01, device=device)

# Load the MNIST training and test data
transform = transforms.Compose([transforms.ToTensor(), transforms.Lambda(lambda x: x.view(-1))])
mnist_train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(mnist_train_data, batch_size=10, shuffle=True)

print("Training RBM4...")
rbm4.train(train_loader, epochs=100)
print("Training RBM8...")
rbm8.train(train_loader, epochs=100)

Once we have the trained RBMs we need to create an encoding of MNIST dataset using their hidden neurons. We encode the MNIST twice, one time for each RBM and we will use these encodings to train a couple of simple classifiers. Their performances will reflect the RBMs' encodings quality.

The following code block will show the implementation of the encoding function and its call. The code block after will show the implementation of the classifier and its training loop, with its training phase and evaluation phase.

In [ ]:
def encode_dataset(rbm, data_loader, device):
    # Lists to store encodings and labels
    encodings = []
    labels = []

    # Iterate through the dataset
    for batch, batch_labels in data_loader:
        batch = batch.to(device)  # Move to the correct device
        # Compute hidden neuron activations for the entire batch
        hidden_activations = rbm.visible_to_hidden(batch)
        encodings.append(hidden_activations.cpu())  # Store encodings
        labels.append(batch_labels)  # Store labels

    # Concatenate all batches into a single tensor
    encodings = torch.cat(encodings, dim=0)
    labels = torch.cat(labels, dim=0)

    return encodings, labels

############################################################

mnist_test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(mnist_test_data, batch_size=10, shuffle=False)

rbm4_train_encodings, rbm4_train_labels = encode_dataset(rbm4, train_loader, device)
rbm4_test_encodings, rbm4_test_labels = encode_dataset(rbm4, test_loader, device)

rbm8_train_encodings, rbm8_train_labels = encode_dataset(rbm8, train_loader, device)
rbm8_test_encodings, rbm8_test_labels = encode_dataset(rbm8, test_loader, device)

In [ ]:
import torch.nn as nn
import torch.optim as optim


class MNISTClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(MNISTClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        return self.fc2(x)
    

def train_classifier_on_mnist(train_loader, input_dim, num_classes, device, num_epochs=40, lr=0.001):
    # Define the classifier
    classifier = MNISTClassifier(input_dim, num_classes).to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(classifier.parameters(), lr=lr)

    # Training loop
    print("Training classifier...")
    for epoch in range(num_epochs):
        classifier.train()
        epoch_loss = 0
        for batch, labels in train_loader:
            batch, labels = batch.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = classifier(batch)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}")

    return classifier


def evaluate_classifier(classifier, test_loader, device):
    classifier.eval()  # Set the classifier to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation for evaluation
        for batch, labels in test_loader:
            batch, labels = batch.to(device), labels.to(device)
            outputs = classifier(batch)
            _, predicted = torch.max(outputs, 1)  # Get the class with the highest score
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total
    return accuracy


#############################################################################